In [1]:
# !pip install tensorflow
# !pip install imutils
# !pip install opencv-python
# !pip install imageio

In [2]:
from tensorflow import keras
from imutils import paths
from keras import layers
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os
import re
from keras.applications import InceptionV3
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
from keras.layers import GRU, Dense, Dropout, GlobalAveragePooling2D, Bidirectional, LSTM, Reshape
from keras.optimizers import Adam
from keras.utils import to_categorical



In [6]:
# Check if directories exist
train_dir = 'Football_video_dataset/train'
test_dir = 'Football_video_dataset/test'
if not os.path.exists(train_dir) or not os.path.exists(test_dir):
    raise FileNotFoundError("Training or testing directories do not exist.")

label_types = os.listdir('Football_video_dataset/train')
print (label_types)


['Loss', 'Goal', 'Happy']


In [7]:
dataset_path = os.listdir('Football_video_dataset/train')
print(dataset_path)

video_list = []

for item in dataset_path:

 # Get all the file names
 all_video_list = os.listdir('Football_video_dataset/train' + '/' +item)

 # Add them to the list
 for video in all_video_list:
    video_list.append((item, str('Football_video_dataset/train' + '/' +item) + '/' + video))

# Build a dataframe
train_df = pd.DataFrame(data=video_list, columns=['tag', 'video_name'])
print(train_df.head())
print(train_df.tail())
df = train_df.loc[:,['video_name','tag']]
df.to_csv('train.csv')


['Loss', 'Goal', 'Happy']
    tag                                        video_name
0  Loss  Football_video_dataset/train/Loss/Loss (128).avi
1  Loss   Football_video_dataset/train/Loss/Loss (64).avi
2  Loss   Football_video_dataset/train/Loss/Loss (87).avi
3  Loss   Football_video_dataset/train/Loss/Loss (62).avi
4  Loss   Football_video_dataset/train/Loss/Loss (66).avi
       tag                                         video_name
313  Happy  Football_video_dataset/train/Happy/Happy (94).mp4
314  Happy  Football_video_dataset/train/Happy/Happy (87).mp4
315  Happy  Football_video_dataset/train/Happy/Happy (37).mp4
316  Happy  Football_video_dataset/train/Happy/Happy (27).mp4
317  Happy  Football_video_dataset/train/Happy/Happy (12).mp4


In [8]:
dataset_path = os.listdir('Football_video_dataset/test')
print(dataset_path)
video_list = []

for item in dataset_path:
 # Get all the file names
 all_video_list = os.listdir('Football_video_dataset/test' + '/' +item)

 # Add them to the list
 for video in all_video_list:
    video_list.append((item, str('Football_video_dataset/test' + '/' +item) + '/' + video))

# Build a dataframe
test_df = pd.DataFrame(data=video_list, columns=['tag', 'video_name'])
print(test_df.head())
print(test_df.tail())

df = test_df.loc[:,['video_name','tag']]
df.to_csv('test.csv')


['Loss', 'Goal', 'Happy']
    tag                                       video_name
0  Loss    Football_video_dataset/test/Loss/Loss (5).avi
1  Loss  Football_video_dataset/test/Loss/Loss (125).avi
2  Loss  Football_video_dataset/test/Loss/Loss (122).avi
3  Loss  Football_video_dataset/test/Loss/Loss (120).avi
4  Loss    Football_video_dataset/test/Loss/Loss (9).avi
      tag                                         video_name
73  Happy  Football_video_dataset/test/Happy/Happy (123).mp4
74  Happy  Football_video_dataset/test/Happy/Happy (128).mp4
75  Happy  Football_video_dataset/test/Happy/Happy (119).mp4
76  Happy  Football_video_dataset/test/Happy/Happy (115).mp4
77  Happy  Football_video_dataset/test/Happy/Happy (129).mp4


In [9]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")



Total videos for training: 318
Total videos for testing: 78


In [10]:
# Function to load and preprocess video frames and save them to a folder

def load_video_frames(video_path, output_folder, capture_percentage=0.6):
  cap = cv2.VideoCapture(video_path)
  total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # Get total video frames (estimate)
  frame_count = 0

  try:
    if total_frames > 0:  # Check for valid total frames
      # Calculate capture interval based on total frames and percentage
      capture_interval = int(total_frames / (capture_percentage * total_frames))

      while True:
        ret, frame = cap.read()
        if not ret:
          break

        # Capture logic based on capture interval
        if frame_count % capture_interval == 0:
          frame_path = os.path.join(output_folder, f"frame_{frame_count}.jpg")
          cv2.imwrite(frame_path, frame)
          frame_count += 1

  except Exception as e:
    print(f"Error loading video {video_path}: {e}")
  finally:
    cap.release()

In [11]:
# Function to save preprocess video frames to a folder
def load_data(video_dir="Football_video_dataset", train_folder="train", test_folder="test"):
    train_labels = {}
    train_df = pd.read_csv(os.path.join("train.csv"))
    for _, row in train_df.iterrows():
        train_labels[row['video_name']] = row['tag']

    test_labels = {}
    test_df = pd.read_csv(os.path.join( "test.csv"))
    for _, row in test_df.iterrows():
        test_labels[row['video_name']] = row['tag']

    X_train, y_train = [], []
    X_test, y_test = [], []

    for class_folder in os.listdir(os.path.join(video_dir, train_folder)):
        class_path = os.path.join(video_dir, train_folder, class_folder)
        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith((".mp4", ".avi")):
                    video_path = os.path.join(class_path, filename)
                    output_folder = os.path.join(video_dir, train_folder + "_frames", class_folder, os.path.splitext(filename)[0])
                    os.makedirs(output_folder, exist_ok=True)
                    load_video_frames(video_path, output_folder)
                    X_train.append(output_folder)
                    y_train.append(train_labels.get(filename, None))

    for class_folder in os.listdir(os.path.join(video_dir, test_folder)):
        class_path = os.path.join(video_dir, test_folder, class_folder)
        if os.path.isdir(class_path):
            for filename in os.listdir(class_path):
                if filename.endswith((".mp4", ".avi")):
                    video_path = os.path.join(class_path, filename)
                    output_folder = os.path.join(video_dir, test_folder + "_frames", class_folder, os.path.splitext(filename)[0])
                    os.makedirs(output_folder, exist_ok=True)
                    load_video_frames(video_path, output_folder)
                    X_test.append(output_folder)
                    y_test.append(test_labels.get(filename, None))

    return (X_train, y_train), (X_test, y_test)

# Load data
(X_train, y_train), (X_test, y_test) = load_data()



In [12]:

# Hyperparameters
EPOCHS = 10
BATCH_SIZE = 128


# Data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory="Football_video_dataset/train_frames",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory="Football_video_dataset/test_frames",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Define InceptionV3 model with transfer learning
base_model = InceptionV3(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Reshape((1, 2048))(x)
x = Bidirectional(LSTM(128, return_sequences=True))(x)
x = Dropout(0.5)(x)  # Add dropout to combat overfitting
x = Bidirectional(LSTM(128))(x)
x = Dropout(0.5)(x)

# Output layer
predictions = Dense(3, activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers in base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# ModelCheckpoint callback to save the best model based on validation loss
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# EarlyStopping callback to stop training if validation loss doesn't improve
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

# Train the model
history = model.fit(train_generator, epochs=EPOCHS, validation_data=test_generator, callbacks=[checkpoint, early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Save the trained model
model.save("football_video_model.keras")

Found 53381 images belonging to 3 classes.
Found 6780 images belonging to 3 classes.
87910968/87910968 [==============================] - 1s 0us/step
Epoch 1/10
418/418 [==============================] - ETA: 0s - loss: 0.1508 - accuracy: 0.9434
Epoch 1: val_loss improved from inf to 0.48449, saving model to best_model.keras
418/418 [==============================] - 351s 786ms/step - loss: 0.1508 - accuracy: 0.9434 - val_loss: 0.4845 - val_accuracy: 0.8906
Epoch 2/10
418/418 [==============================] - ETA: 0s - loss: 0.0599 - accuracy: 0.9801
Epoch 2: val_loss improved from 0.48449 to 0.39243, saving model to best_model.keras
418/418 [==============================] - 288s 689ms/step - loss: 0.0599 - accuracy: 0.9801 - val_loss: 0.3924 - val_accuracy: 0.9083
Epoch 3/10
418/418 [==============================] - ETA: 0s - loss: 0.0425 - accuracy: 0.9859
Epoch 3: val_loss improved from 0.39243 to 0.36187, saving model to best_model.keras
418/418 [==============================] 